# Starbucks: Analyze-a-Coffee

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

# magic word for producing visualizations in notebook
%matplotlib inline

import plotly.plotly as py #for creating interactive data visualizations
import plotly.graph_objs as go
import plotly.tools as tls
py.sign_in('', '') #API key has been removed for security
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot #to work with data visualization offline
init_notebook_mode(connected=True)
import cufflinks as cf #connects Plotly with pandas to produce the interactive data visualizations
cf.go_offline()

from IPython.display import Image

In [2]:
# reading the json files
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)
profile = pd.read_json('profile.json', orient='records', lines=True)
transcript = pd.read_json('transcript.json', orient='records', lines=True)

---
---
---
# 1. Data Wrangling

## 1.1 Data Sets
The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

### <font color=blue> 1.1.1 Portfolio

In [6]:
# printing the portfolio data

print(f"Number of offers: {portfolio.shape[0]}")
print(f"Number of variables: {portfolio.shape[1]}")

portfolio

Number of offers: 10
Number of variables: 6


,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


Here is the schema and explanation of each variable in the file:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

In [9]:
# overall info about the portfolio data

portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
channels      10 non-null object
difficulty    10 non-null int64
duration      10 non-null int64
id            10 non-null object
offer_type    10 non-null object
reward        10 non-null int64
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


In [10]:
portfolio.describe()

,difficulty,duration,reward
count,10.000000,10.000000,10.000000
mean,7.700000,6.500000,4.200000
std,5.831905,2.321398,3.583915
min,0.000000,3.000000,0.000000
25%,5.000000,5.000000,2.000000
50%,8.500000,7.000000,4.000000
75%,10.000000,7.000000,5.000000
max,20.000000,10.000000,10.000000


The first look at the portfolio indicates that we will have to split the channels in the cleanning process.

---
### <font color=blue> 1.1.2 Profile

In [7]:
# printing the profile data

print(f"Number of customers: {profile.shape[0]}")
print(f"Number of variables: {profile.shape[1]}")
profile.sample(10)

Number of customers: 17000
Number of variables: 5


,age,became_member_on,gender,id,income
4790,57,20180531,F,ccbb447a19e04f7fa7a36dae6991b736,61000.0
5710,69,20160103,F,b13e68942b4a47f6b0aa518eb69adb47,62000.0
12134,73,20150929,F,a10162dac5614ce28bc0e2d154e2224c,87000.0
11690,52,20150608,M,d31ccd5fc13d49f682db3b7fde074bd5,71000.0
7537,40,20150426,M,c9f5e6b7918540d692179bf5b1608935,66000.0
13942,50,20160918,M,c676b248358f48ac86a47d2fb190ffef,92000.0
8147,78,20160516,M,625b2edda53f4e5db514565ee8c11e17,111000.0
11126,21,20160711,M,b80fdfaf366941fd972f5802408c1cb8,52000.0
6424,118,20170306,None,fec978ba6c9b4c719dd401cba3a077cb,NaN
8412,21,20160526,M,57b6aab7967146d88ff5645f56e2dd07,48000.0


Here is the schema and explanation of each variable in the file:

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

In [12]:
# overall info about the profile data

profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
age                 17000 non-null int64
became_member_on    17000 non-null int64
gender              14825 non-null object
id                  17000 non-null object
income              14825 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.1+ KB


We can see that we have 17000 customers in our data. However, gender and income variables have some nulls values which will be investigated next. 

In [13]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


The age variable contains a maximum value of 118 years old, which is considered an unusual value that will be investigated even further.

In [8]:
# checking the number of null values

profile.isnull().sum()

age                    0
became_member_on       0
gender              2175
id                     0
income              2175
dtype: int64

In [31]:
# printing the number of nulls values for the gender column by age

print(f"Number of null values: {profile.age[profile.gender.isnull()].value_counts().iloc[0]}")
print(f"Number of unqiue customers with null values: {profile.age[profile.gender.isnull()].nunique()}")
print(f"The age of the unique customers where there are nulls: {profile.age[profile.gender.isnull()][0]}")


Number of null values: 2175
Number of unqiue customers with null values: 1
The age of the unique customers where there are nulls: 118


As expected, all the nulls in the profile data are part of the customers who have '118' as an age value. Therefore, the rows that have the value 118 will be dropped as a part of the cleanning process of the profile file.

---
### <font color=blue> 1.1.3 Transcript

In [33]:
# printing the transcript data

print(f"Number of transcripts: {transcript.shape[0]}")
print(f"Number of variables: {transcript.shape[1]}")
transcript.sample(5)

Number of transcripts: 306534
Number of variables: 4


,event,person,time,value
15652,offer completed,9ab83b0a9fe74e4ead33c741bcc4c2ea,6,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...
267965,offer viewed,da53066ed4d84acfacb1d696b8516622,588,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
55420,offer received,153f7ce19d8f4c8294fdc8d758c880c7,168,{'offer id': '3f207df678b143eea3cee63160fa8bed'}
98044,offer viewed,f4994572197d4310bf922576d5a8d132,264,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
126843,offer completed,ab7e052ddbc040938db0e74a56d97b0f,336,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...


Here is the schema and explanation of each variable in the files:

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [18]:
# overall info about the transcript data

transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
event     306534 non-null object
person    306534 non-null object
time      306534 non-null int64
value     306534 non-null object
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [19]:
transcript.time.describe()

count    306534.000000
mean        366.382940
std         200.326314
min           0.000000
25%         186.000000
50%         408.000000
75%         528.000000
max         714.000000
Name: time, dtype: float64

The transcripts data includes an important variable, **time**, that will be used to make sure each event, or transaction was made within the **duration** of an offer. All of these cleanning process will be applied next.

---
---
---
## 1.2 Data Cleaning

### <font color=blue> 1.2.1 Portfolio

In [35]:
# creating a new copy for cleaning purposes

portfolio_clean = portfolio.copy()

In [36]:
# overall view of the portfolio data before cleaning

portfolio_clean.sample(3)

,channels,difficulty,duration,id,offer_type,reward
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10


In [37]:
# showing the channels available before splitting them to 4 columns

portfolio_clean.channels.sample(5)

2            [web, email, mobile]
7         [email, mobile, social]
8    [web, email, mobile, social]
5    [web, email, mobile, social]
9            [web, email, mobile]
Name: channels, dtype: object

In [38]:
# splitting the channels

def col_split(df, column):
    splits = []
    for s in df[column]:
        for i in s:
            if i not in splits:
                splits.append(i)
    for split in splits:
        df[split] = df[column].apply(lambda x: 1 if split in x else 0)
        
    df.drop([column], axis=1, inplace=True) 
    
    return splits

col_split(portfolio_clean ,'channels')

['email', 'mobile', 'social', 'web']

In [39]:
# overall view of the portfolio data after the split

portfolio_clean.sample(3)

,difficulty,duration,id,offer_type,reward,email,mobile,social,web
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,1,1,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,1,0,0,1
9,10,7,2906b810c7d4411798c6938adc9daaa5,discount,2,1,1,0,1


We have just split the channels where each channel become a variable and has the value 1 if it has been used as a channel for a specific offer whereas the value 0 means it hasn't been used. 

In [40]:
# showing the type of each column

portfolio_clean.dtypes

difficulty     int64
duration       int64
id            object
offer_type    object
reward         int64
email          int64
mobile         int64
social         int64
web            int64
dtype: object

In [41]:
# converting the duration column to be in hours instead of days
# to be compared with the times columns in the transcript file

portfolio_clean['duration'] = portfolio_clean['duration'] * 24

As mentioned eariler, the duration variable will play a major role in the cleanning process where it will be compared to the time of each transcript provided. Thus, it will be converted to be in hours as the time variables does.

In [42]:
# renaming the columns for the portfolio data

portfolio_clean.rename(columns={'difficulty': 'difficulty($)', 
                                'duration': 'duration(hours)', 
                                'id': 'offer_id'}, inplace=True)

In [43]:
# overall view of the portfolio data after renaming the columns

portfolio_clean.head(3)

,difficulty($),duration(hours),offer_id,offer_type,reward,email,mobile,social,web
0,10,168,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,1,1,0
1,10,120,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,1,1,1
2,0,96,3f207df678b143eea3cee63160fa8bed,informational,0,1,1,0,1


In [44]:
# ordering the columns for the final look at the portfolio data after being clean 

portfolio_clean = portfolio_clean[['offer_id', 
                                   'offer_type', 
                                   'difficulty($)', 
                                   'duration(hours)', 
                                   'reward', 
                                   'email', 
                                   'mobile', 
                                   'social', 
                                   'web']]
portfolio_clean.head(3)

,offer_id,offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,168,10,1,1,1,0
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,120,10,1,1,1,1
2,3f207df678b143eea3cee63160fa8bed,informational,0,96,0,1,1,0,1


---
### <font color=blue> 1.2.2 Profile

In [45]:
# creating a new copy for cleaning purposes

profile_clean = profile.copy()

In [46]:
# overall view of the profile data before cleaning

profile_clean.head(3)

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN


In [47]:
# showing the type of each column

profile_clean.dtypes

age                   int64
became_member_on      int64
gender               object
id                   object
income              float64
dtype: object

In [48]:
# converting the 'became_member_on' column to date
# adding a period of the membership by month

profile_clean['membership_start'] = profile_clean.became_member_on.apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
profile_clean['membership_period'] = profile_clean['membership_start'].dt.to_period('M')
profile_clean.drop(['became_member_on'], axis=1, inplace=True)

The variable "became_member_on" could be used for finding trends with respect to the time most customers have joined Starbucks App. Therefore, the column was converted to be a date instead of int and then a membership_period by Month was created where it shows the month and year when the customer become a member.

In [49]:
# overall view of the profile data after applying some cleaning

profile_clean.head(3)

,age,gender,id,income,membership_start,membership_period
0,118,None,68be06ca386d4c31939f3a4f0e3dd783,NaN,2017-02-12,2017-02
1,55,F,0610b486422d4921ae7d2bf64640c50b,112000.0,2017-07-15,2017-07
2,118,None,38fe809add3b4fcf9315a9694bb96ff5,NaN,2018-07-12,2018-07


In [50]:
# renaming the id column for the profile data

profile_clean.rename(columns={'id': 'customer_id'}, inplace=True)

In [51]:
# listing the customers_ids whose age value is '118',
# which accounts for the all the nulls values

age_118 = profile_clean[profile_clean['age'] == 118]
list(age_118.customer_id)

['68be06ca386d4c31939f3a4f0e3dd783',
 '38fe809add3b4fcf9315a9694bb96ff5',
 'a03223e636434f42ac4c3df47e8bac43',
 '8ec6ce2a7e7949b1bf142def7d0e0586',
 '68617ca6246f4fbc85e91a2a49552598',
 '8974fc5686fe429db53ddde067b88302',
 'c4863c7985cf408faee930f111475da3',
 '148adfcaa27d485b82f323aaaad036bd',
 '744d603ef08c4f33af5a61c8c7628d1c',
 '2b826eba31074a059d63b0ae8f50b7d5',
 '65aba5c617294649aeb624da249e1ee5',
 '4beeb3ed64dd4898b0edf2f6b67426d3',
 '25c906289d154b66bf579693f89481c9',
 '5b8f1f0e0930464f93bb55e59b5a04e0',
 '8be4463721e14d7fa600686bf8c8b2ed',
 '81ac76e4f2fe4a6bb5ebdc0f19cecc56',
 '020d3a52b9ce432ea5f98fc58e9bfcc4',
 '4be1127413db496d8103c783ae72058b',
 '9d8d6bb2fc544a9c802888971ee3e8b1',
 'a89752cdfdf24779aa22faa2bc24e7c2',
 '53bac762af27471eb294ff89ca1bae2c',
 '488773cae11f4b71875bfccd4e3e99f2',
 '182e68952b474700b171e21b85c258ac',
 '5e9c83e8ab4d4e9ca83f3256d83ca756',
 'c099206f76b1414db7552f163520053c',
 'e6d3eebb535e44aca1323edde5c346fa',
 'baba98dba43e42ae8e86cbd70725e201',
 

In [53]:
# making sure the profile data doesn't contains the customers_ids whose age value is '118',
# which accounts for the all the nulls values

profile_clean = profile_clean[profile_clean.age != 118]
profile_clean.head(3)

,age,gender,customer_id,income,membership_start,membership_period
1,55,F,0610b486422d4921ae7d2bf64640c50b,112000.0,2017-07-15,2017-07
3,75,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017-05-09,2017-05
5,68,M,e2127556f4f64592b11af22de27a7932,70000.0,2018-04-26,2018-04


In [54]:
print(f"Number of customers after cleaning: {profile_clean.shape[0]}")
print(f"Number of variables: {profile_clean.shape[1]}")

Number of customers after cleaning: 14825
Number of variables: 6


We talked about the customers who have nulls, and then found out that all nulls can be dropped when the rows with age = 118 are dropped, which what we have just done. We recreated the profile data to exclude the rows with age 118. As a result, the prfile dataset contain information about 14825 instead of 17000 customers

In [56]:
# checking the number of null values after dropping the customers ids with nulls income and gender

profile_clean.isnull().sum()

age                  0
gender               0
customer_id          0
income               0
membership_start     0
membership_period    0
dtype: int64

In [57]:
# ordering the columns for the final look at the profile data after being clean 

profile_clean = profile_clean[['customer_id', 
                               'age', 
                               'gender', 
                               'income', 
                               'membership_start', 
                               'membership_period']]
profile_clean.head(3)

,customer_id,age,gender,income,membership_start,membership_period
1,0610b486422d4921ae7d2bf64640c50b,55,F,112000.0,2017-07-15,2017-07
3,78afa995795e4d85b5d9ceeca43f5fef,75,F,100000.0,2017-05-09,2017-05
5,e2127556f4f64592b11af22de27a7932,68,M,70000.0,2018-04-26,2018-04


---
### <font color=blue> 1.2.3 Transcript

In [58]:
# creating a new copy for cleaning purposes

transcript_clean = transcript.copy()

In [59]:
# overall view of the transcript data before cleaning

transcript_clean.head(3)

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}


In [60]:
# showing the type of each column

transcript_clean.dtypes

event     object
person    object
time       int64
value     object
dtype: object

In [61]:
# showing the values available before splitting them to 
# 2 columns: record (offer id or transactions) and record_value (amount or id)

transcript_clean.value.sample(10)

209187     {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
188496                                    {'amount': 15.18}
42092                                      {'amount': 0.45}
280647                                    {'amount': 14.48}
298891                                     {'amount': 3.37}
260354     {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
211678     {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
267226     {'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
304240    {'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...
305331                                     {'amount': 0.17}
Name: value, dtype: object

While the time variable would play an important role during the analysis, we should first split the values we have either offer_id with its record or transaction with the amount. The next function will do the magic!

In [62]:
# splitting the value column

def df_values(df=transcript_clean):
    df['record'] = df.value.apply(lambda x: list(x.keys())[0])
    df['record_value'] = df.value.apply(lambda x: list(x.values())[0])
        
    df.drop(['value'], axis=1, inplace=True) 
    
    return None

df_values()

In [63]:
# renaming the person and time columns for the transcript data

transcript_clean.rename(columns={'person': 'customer_id', 
                                 'time': 'time(hours)'}, inplace=True)
transcript_clean.head(3)

,event,customer_id,time(hours),record,record_value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,offer id,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,offer received,e2127556f4f64592b11af22de27a7932,0,offer id,2906b810c7d4411798c6938adc9daaa5


In [64]:
# ordering the columns for the final look at the transcript data after being clean

transcript_clean = transcript_clean[['customer_id', 
                                     'event', 
                                     'record', 
                                     'record_value', 
                                     'time(hours)']]
transcript_clean = transcript_clean[~transcript_clean.customer_id.isin(age_118.customer_id)]
transcript_clean.head(3)

,customer_id,event,record,record_value,time(hours)
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9,0
2,e2127556f4f64592b11af22de27a7932,offer received,offer id,2906b810c7d4411798c6938adc9daaa5,0
5,389bc3fa690240e798340f5a15918d5c,offer received,offer id,f19421c1d4aa40978ebb69ca19b0e20d,0


After splitting the ids and the transactions, we are goining to create two datasets: offers that contains the information where the record is offer_id and the record value is the id, and transactions which include the amount of each transaction made by a customer.

In [65]:
print(f"Number of transcripts after cleaning: {transcript_clean.shape[0]}")
print(f"Number of variables: {transcript_clean.shape[1]}")

Number of transcripts after cleaning: 272762
Number of variables: 5


In [66]:
# splitting the transcript dataset to 2 datasets; offers and transactions

transactions = transcript_clean[transcript_clean.event == 'transaction']
offers = transcript_clean[transcript_clean.event != 'transaction']

In [67]:
# overall view of the offers data

offers.sample(3)

,customer_id,event,record,record_value,time(hours)
118220,48609f44985a47159c17a4d7c82f5a39,offer received,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9,336
255131,21928f294b2b414c8ecd694538393567,offer received,offer id,ae264e3637204a6fb9bb56bc8210ddfd,576
252571,fb16abdfaa1d421d9e11f1ac5c8e80df,offer received,offer id,9b98b8c7a33c4b65b9aebfe6a799e6d9,576


In [68]:
# renaming the record_value

offers.rename(columns={'record_value': 'offer_id'}, inplace=True)
offers.drop(['record'], axis=1, inplace=True) 

In [70]:
print(f"Number of offers after cleaning: {offers.shape[0]}")
print(f"Number of variables: {offers.shape[1]}")

offers.sample(3)

Number of offers after cleaning: 148805
Number of variables: 4


,customer_id,event,offer_id,time(hours)
254414,9cff812d0f8f426a8189648d3c476411,offer received,2906b810c7d4411798c6938adc9daaa5,576
10506,11642a741bf64c9f917cb5439cb4d580,offer received,f19421c1d4aa40978ebb69ca19b0e20d,0
166862,63d80d59c27a4f89bc3a9d5ad28bebe6,offer completed,2906b810c7d4411798c6938adc9daaa5,408


In [71]:
# converting the record value for the transactions to be numerical

transactions['record_value'] = pd.to_numeric(transactions['record_value'])
transactions.dtypes

customer_id      object
event            object
record           object
record_value    float64
time(hours)       int64
dtype: object

In [72]:
# overall view of the transactions data

transactions.sample(3)

,customer_id,event,record,record_value,time(hours)
305626,e300729d05c544cfad882e049e867a70,transaction,amount,3.88,714
304257,a19ede978c814ac28267c51c93c1cd2c,transaction,amount,25.32,702
295329,59cfbf8463ab4b03a374198c1fd765e3,transaction,amount,25.97,666


In [73]:
# renaming the record_value

transactions.rename(columns={'record_value': 'transaction_amount'}, inplace=True)
transactions.drop(['record', 'event'], axis=1, inplace=True) 

In [74]:
print(f"Number of transactions after cleaning: {transactions.shape[0]}")
print(f"Number of variables: {transactions.shape[1]}")

transactions.sample(3)

Number of transactions after cleaning: 123957
Number of variables: 3


,customer_id,transaction_amount,time(hours)
37436,e3b6fba0f8f54dd4b06f045dcbff29bd,12.00,78
92373,773b9b57713f470fbfe74ab32a41a945,7.32,240
53067,f6cff88b6ad24c2aab172b9b8ab17edd,29.44,162


---
# 2. Data Exploration

## 2.1 Part 1

### <font color=blue> 2.1.1 Profile

In [75]:
# overall view of the profile data after being clean

profile_clean.sample(3)

,customer_id,age,gender,income,membership_start,membership_period
1559,5a5f5e54f76249b38c884d35a4cca242,57,M,66000.0,2017-08-01,2017-08
865,7195944f0cc34115b0a5e7b4a62055f2,47,F,77000.0,2016-03-06,2016-03
6354,f4fa477e66204420b0c0b90448596a46,84,F,106000.0,2017-08-16,2017-08


In [76]:
# describing the values of the income column

profile_clean.income.describe()

count     14825.000000
mean      65404.991568
std       21598.299410
min       30000.000000
25%       49000.000000
50%       64000.000000
75%       80000.000000
max      120000.000000
Name: income, dtype: float64

In [77]:
# describing the values of the age column

profile_clean.age.describe()

count    14825.000000
mean        54.393524
std         17.383705
min         18.000000
25%         42.000000
50%         55.000000
75%         66.000000
max        101.000000
Name: age, dtype: float64

In [58]:
# plotting the numbers and percentages of customers by gender

gender = profile_clean.gender.value_counts()
labels = gender.index
values = gender
colors = ['cornflowerblue', 'pink', 'mediumblue']


trace1 = go.Pie(labels=labels, values=round(100*values/values.sum(), 2),
               text=values,
               textfont=dict(size=20),
               hoverinfo='skip',
               hole=0.9,
               showlegend=False,
               opacity=0.6,
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))


trace2 = go.Bar(
    x=['Male'],
    y=[values[0]],
    name='Male',
    marker=dict(
        color='cornflowerblue',
        line=dict(
            color='cornflowerblue',
            width=1.5,
        )
    ),
    opacity=0.6
)

trace3 = go.Bar(
    x=['Female'],
    y=[values[1]],
    name='Female',
    marker=dict(
        color='pink',
        line=dict(
            color='pink',
            width=1.5,
        )
    ),
    opacity=0.6
)

trace4 = go.Bar(
    x=['Other'],
    y=[values[2]],
    name='Other',
    marker=dict(
        color='mediumblue',
        line=dict(
            color='mediumblue',
            width=1.5,
        )
    ),
    opacity=0.6
)

data1 = go.Data([trace1, trace2, trace3, trace4])

layout = go.Layout(
    title='Number of Customers by Gender',
    xaxis=dict(
        title='Gender',
        domain=[0.4, 0.6]
    ),
    yaxis=dict(
        title='Count',
        domain=[0.4, 0.7]
    )
)

fig = go.Figure(data=data1, layout=layout)
py.iplot(fig, filename='percentage of customer by gender')

/Users/Saleh/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/Users/Saleh/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



The dount chart illustrates that female customers account for 41.30% and male customers account for 57.20% of the data overall whether they all completed an over, made a transaction before the end of an offer or not. Also, customers who chose other as gender account only for 1.43%.

In [59]:
# plotting a gender comparsion with respect to income and age

def array(df, gen, col):
    arr = df[df.gender == gen]
    arr = np.array(arr[col])
    
    return arr

def gender_comparsion(df, col, plot_name, avg):
    x_data = ['Male', 'Female', 'Other']

    y0 = array(df, 'M', col)
    y1 = array(df, 'F', col)
    y2 = array(df, 'O', col)

    y_data = [y0,y1,y2]

    colors = ['cornflowerblue', 'pink', 'mediumblue']

    traces = []

    for xd, yd, cls in zip(x_data, y_data, colors):
            traces.append(go.Box(
                y=yd,
                name=xd,
                boxmean=True,
                boxpoints='all',
                jitter=0.5,
                whiskerwidth=0.2,
                marker=dict(
                    size=2,
                    color=cls,
                ),
                line=dict(color=cls,
                          width=1),
            ))

    layout = {
    'title': plot_name,               
    'shapes': [
        # Line Horizontal, average
        {
            'type': 'line',
            'xref': 'paper',
            'x0': 0,
            'y0': avg,
            'x1': 1,
            'y1': avg,
            'line': {
                'color': 'black',
                'width': 1,
                'dash': 'dashdot',
            }
        }]}
    
    layout.update(dict(
    annotations=[go.Annotation(text="Overall Average", y=avg)]))
    
    fig = go.Figure(data=traces, layout=layout)
    plot = py.iplot(fig, filename=plot_name)
    
    return plot

In [60]:
gender_comparsion(profile_clean, 'income', 'Income Comparsion', 64000)

/Users/Saleh/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:144: DeprecationWarning:

plotly.graph_objs.Annotation is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Annotation
  - plotly.graph_objs.layout.scene.Annotation




We can see that, on average, male customers make less than female customers do in a year. In fact, female customers make more than the average income per year for all gender types.

In [61]:
gender_comparsion(profile_clean, 'age', 'Age Comparsion', 55)

The box plots that elaborate a summary of the age distribution by gender. They show that, on average, female customers are older than male customers.  

In [81]:
# grouping the data by income and then reporting the average number of customers per icnome

print(f"The average number of customers per icnome rate per year: {round(profile_clean.groupby('income')['customer_id'].count().mean(),0)}")

The average number of customers per icnome rate per year: 163.0


In [256]:
# plotting the Income Distribution by Gender

dis = profile_clean.drop_duplicates(subset='customer_id', keep='first')
female_income = dis[dis['gender'] == 'F']
male_income = dis[dis['gender'] == 'M']
other_income = dis[dis['gender'] == 'O']

x1 = female_income['income']
x2 = male_income['income']
x3 = other_income['income']

trace1 = go.Histogram(
    x=x1,
    name='Female',
    opacity=0.6,
    nbinsx = 91,
    marker=dict(
        color='pink')
)
    
trace2 = go.Histogram(
    x=x2,
    name='Male',
    opacity=0.6,
    nbinsx = 91,
    marker=dict(
        color='cornflowerblue')
)

trace3 = go.Histogram(
    x=x3,
    name='Other',
    opacity=0.6,
    nbinsx = 91,
    marker=dict(
        color='mediumblue')
)

data3 = [trace1, trace2, trace3]

updatemenus = list([
    dict(active=0,
         buttons=list([
             dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True]},
                          {'title': 'Income Distribution by Gender'}]),
            dict(label = 'Female',
                  method = 'update',
                  args = [{'visible': [True, False, False]},
                          {'title': 'Income Distribution of Male Customers'}]),
            dict(label = 'Male',
                  method = 'update',
                  args = [{'visible': [False, True, False]},
                          {'title': 'Income Distribution of Female Customers'}])

        ]),
    )
])

layout = go.Layout(dict(updatemenus=updatemenus,
                  barmode='stack',
                  bargap=0.2,
                  title = 'Income by Gender',
                  xaxis=dict(
                      title='Income($)'), 
                  yaxis=dict(
                      title='Number of Customers')))

fig = go.Figure(data=data3, layout=layout)
py.iplot(fig, filename='Income Distribution by Gender')

Here, the plot clarifies that, overall, the majority of our customers make between 50K and 75K. The average number of customer per income rate per year is 163 customers. Thus, while the minority are making more than 100K per year, some customers make less than 50K and the other make between 76K and 100K. Overall, we can see that most of the female customers make between 60K - 75K, and most of the make customers make 45K - 60K.

---
### <font color=blue> 2.1.2 Portfolio

In [82]:
# overall view of the portfolio data after being clean

portfolio_clean

,offer_id,offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,168,10,1,1,1,0
1,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,120,10,1,1,1,1
2,3f207df678b143eea3cee63160fa8bed,informational,0,96,0,1,1,0,1
3,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,168,5,1,1,0,1
4,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20,240,5,1,0,0,1
5,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,7,168,3,1,1,1,1
6,fafdcd668e3743c1bb461111dcafc2a4,discount,10,240,2,1,1,1,1
7,5a8bc65990b245e5a138643cd4eb9837,informational,0,72,0,1,1,1,0
8,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,120,5,1,1,1,1
9,2906b810c7d4411798c6938adc9daaa5,discount,10,168,2,1,1,0,1


In [83]:
# showing the number of offers by offer_type

portfolio_clean.groupby('offer_type')['offer_id'].count()

offer_type
bogo             4
discount         4
informational    2
Name: offer_id, dtype: int64

To be used for further analysis, we can that 4 of the offers are BOGO, 4 are Discounts, and 2 are informational that don't require any compleation by customers as well as don't provide any rewards. Hence, we will focus only on the BOGO and Discounts.

---
### <font color=blue> 2.1.3 Transcript

In [84]:
# overall view of the transcript data after being clean

transcript_clean.sample(3)

,customer_id,event,record,record_value,time(hours)
197185,236a66dab68243f5bfde1277386b4e6c,transaction,amount,16.73,486
184292,27cb665ec6754c2b8731f37deefb229c,offer completed,offer_id,f19421c1d4aa40978ebb69ca19b0e20d,444
31006,1f4d50d43cfe4547b3dd37d318a83b0a,transaction,amount,22.92,48


In [85]:
# showing the number of transcripts by event

transcript_clean.event.value_counts()

transaction        123957
offer received      66501
offer viewed        49860
offer completed     32444
Name: event, dtype: int64

In [86]:
# showing the percentage of transcripts by event

transcript_clean['event'].value_counts(normalize = True)

transaction        0.454451
offer received     0.243806
offer viewed       0.182797
offer completed    0.118946
Name: event, dtype: float64

In the transcripts, customers made 123957 transactions overall whether they are considerd on time (made before the end of an offer or not). Therefore, we will clean the transcript dataset even further by making sure we have the transaction made on time while the customer viewed and completed the offer.

In [87]:
# functions to return 3 datasets by event 

def offers_event1(df, ev):
    offers = df[df.event == ev]
    
    return offers

offer_received = offers_event1(offers, 'offer received')
offers_viewed = offers_event1(offers, 'offer viewed')
offers_completed = offers_event1(offers, 'offer completed')

def offers_event2(df):
    df_offers = pd.DataFrame(df.offer_id.value_counts()).reset_index().rename(columns={'index': 'offer_id', 'offer_id': 'count'})
    
    return df_offers

df_received = offers_event2(offer_received)
df_viewed = offers_event2(offers_viewed)
df_completed = offers_event2(offers_completed)

In [89]:
# merging the datasets for each event created from the previous functions to calculate 
# the numbers and percentages of the customers for each offer

offers_count = pd.merge(pd.merge(df_received, df_viewed, on='offer_id'), df_completed, on='offer_id')
offers_count = offers_count.rename(columns={'count_x': 'received', 'count_y': 'viewed','count': 'completed'})
offers_count['offer_views(%)'] = offers_count['viewed']/offers_count['received']*100
offers_count['offer_completion(%)'] = offers_count['completed']/offers_count['received']*100
offers_count['completed(not_viewed)'] = offers_count['completed']-offers_count['viewed']
offers_count

,offer_id,received,viewed,completed,offer_views(%),offer_completion(%),completed(not_viewed)
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,6726,2215,3386,32.931906,50.341957,1171
1,9b98b8c7a33c4b65b9aebfe6a799e6d9,6685,3499,4188,52.341062,62.647719,689
2,ae264e3637204a6fb9bb56bc8210ddfd,6683,5901,3657,88.298668,54.720934,-2244
3,2298d6c36e964ae4a3e7e9706d1fb8c2,6655,6379,4886,95.852742,73.418482,-1493
4,fafdcd668e3743c1bb461111dcafc2a4,6652,6407,5003,96.316897,75.210463,-1404
5,2906b810c7d4411798c6938adc9daaa5,6631,3460,3911,52.179158,58.980546,451
6,4d5c57ea9a6940dd891ad53e9dbe8da0,6593,6329,3310,95.995753,50.204763,-3019
7,f19421c1d4aa40978ebb69ca19b0e20d,6576,6310,4103,95.954988,62.393552,-2207


In [115]:
print(f"The offer with the most completion percentage: {offers_count.loc[offers_count['offer_completion(%)'] == offers_count['offer_completion(%)'].max(), 'offer_id'].iloc[0]}")
print(f"Its completion percentage: {round(offers_count['offer_completion(%)'].max(), 2)}")

The offer with the most completion percentage: fafdcd668e3743c1bb461111dcafc2a4
Its completion percentage: 75.21


An overall analysis of each offer before cleanning the data even more to meet our requirements; a customer viewed and completed an offer by making a transaction. Here we can see than the offer "fafdcd668e3743c1bb461111dcafc2a4" is the most popular offer with around a 75% compleation rate for the customers who received an offer following by "2298d6c36e964ae4a3e7e9706d1fb8c2" with a 73% compleation rate. Also, the less popular offer is "0b1e1539f2cc45b7b9fa7c272da2e1d7" as only 33% of the customers who recevied it have viewed the offer and 50% of the customers who recevied that offer have completed it.

In [117]:
# merging the offers_count data with portfolio data to have a complete info about each offer

offers_comparsion = pd.merge(offers_count, portfolio_clean, on='offer_id')
offers_comparsion

,offer_id,received,viewed,completed,offer_views(%),offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,6726,2215,3386,32.931906,50.341957,1171,discount,20,240,5,1,0,0,1
1,9b98b8c7a33c4b65b9aebfe6a799e6d9,6685,3499,4188,52.341062,62.647719,689,bogo,5,168,5,1,1,0,1
2,ae264e3637204a6fb9bb56bc8210ddfd,6683,5901,3657,88.298668,54.720934,-2244,bogo,10,168,10,1,1,1,0
3,2298d6c36e964ae4a3e7e9706d1fb8c2,6655,6379,4886,95.852742,73.418482,-1493,discount,7,168,3,1,1,1,1
4,fafdcd668e3743c1bb461111dcafc2a4,6652,6407,5003,96.316897,75.210463,-1404,discount,10,240,2,1,1,1,1
5,2906b810c7d4411798c6938adc9daaa5,6631,3460,3911,52.179158,58.980546,451,discount,10,168,2,1,1,0,1
6,4d5c57ea9a6940dd891ad53e9dbe8da0,6593,6329,3310,95.995753,50.204763,-3019,bogo,10,120,10,1,1,1,1
7,f19421c1d4aa40978ebb69ca19b0e20d,6576,6310,4103,95.954988,62.393552,-2207,bogo,5,120,5,1,1,1,1


Final look at the offers analysis with even more details about each offer. It shows that the two most popular offers that have just been discovered are discounts  where all four channels were used reuiqre a mininum of $10 per transaction In fact, more than 95% of the customers who received these offers have viewed the offer through one of the channels.

In [197]:
offers_comparsion.sample(2)

,offer_id,received,viewed,completed,offer_views(%),offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
1,9b98b8c7a33c4b65b9aebfe6a799e6d9,6685,3499,4188,52.341062,62.647719,689,bogo,5,168,5,1,1,0,1
7,f19421c1d4aa40978ebb69ca19b0e20d,6576,6310,4103,95.954988,62.393552,-2207,bogo,5,120,5,1,1,1,1


## 2.2 Part 2


In [118]:
# merging all three datasets to have a complete, clean dataset 

full_data = pd.merge(pd.merge(pd.merge(profile_clean, 
                                  transactions, on='customer_id'), 
                         offers, on='customer_id'), 
                offers_comparsion, on='offer_id')

In [119]:
# locking at the full datasets

full_data

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time(hours)_x,event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
0,0610b486422d4921ae7d2bf64640c50b,55,F,112000.0,2017-07-15,2017-07,21.51,18,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1
1,0610b486422d4921ae7d2bf64640c50b,55,F,112000.0,2017-07-15,2017-07,21.51,18,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1
2,0610b486422d4921ae7d2bf64640c50b,55,F,112000.0,2017-07-15,2017-07,32.28,144,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1
3,0610b486422d4921ae7d2bf64640c50b,55,F,112000.0,2017-07-15,2017-07,32.28,144,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1
4,0610b486422d4921ae7d2bf64640c50b,55,F,112000.0,2017-07-15,2017-07,23.22,528,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1
5,0610b486422d4921ae7d2bf64640c50b,55,F,112000.0,2017-07-15,2017-07,23.22,528,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1
6,78afa995795e4d85b5d9ceeca43f5fef,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1
7,78afa995795e4d85b5d9ceeca43f5fef,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1
8,78afa995795e4d85b5d9ceeca43f5fef,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1
9,78afa995795e4d85b5d9ceeca43f5fef,75,F,100000.0,2017-05-09,2017-05,17.78,144,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,...,62.647719,689,bogo,5,168,5,1,1,0,1


In [120]:
# renaming the time hours of the transactions to be distinguished from the time of offers 
full_data.rename(columns={'time(hours)_x': 'time_of_transaction(hours)'}, inplace=True)

In [121]:
# a function to recreate the ids columns to be easy ids to use and communicate

def id_mapper(df, column):
    coded_dict = dict()
    cter = 1
    id_encoded = []
    
    for val in df[column]:
        if val not in coded_dict:
            coded_dict[val] = cter
            cter+=1
        
        id_encoded.append(coded_dict[val])
    return id_encoded

cid_encoded = id_mapper(full_data, 'customer_id')
full_data['customer_id'] = cid_encoded

oid_encoded = id_mapper(full_data, 'offer_id')
full_data['offer_id'] = oid_encoded

To make even easier to communicate the customers and offers ids, they have been mapped to be represented by numbers instead of hashes or codes.

In [122]:
# locking at the full datasets after recreating the ids

full_data

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
0,1,55,F,112000.0,2017-07-15,2017-07,21.51,18,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
1,1,55,F,112000.0,2017-07-15,2017-07,21.51,18,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
2,1,55,F,112000.0,2017-07-15,2017-07,32.28,144,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
3,1,55,F,112000.0,2017-07-15,2017-07,32.28,144,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
4,1,55,F,112000.0,2017-07-15,2017-07,23.22,528,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
5,1,55,F,112000.0,2017-07-15,2017-07,23.22,528,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
6,2,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
7,2,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
8,2,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
9,2,75,F,100000.0,2017-05-09,2017-05,17.78,144,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1


In [123]:
# making sure the dataset contains only the transactions that appear before the end of an offer

full_data = full_data[full_data['time_of_transaction(hours)'] <= full_data['duration(hours)']]
full_data

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
0,1,55,F,112000.0,2017-07-15,2017-07,21.51,18,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
1,1,55,F,112000.0,2017-07-15,2017-07,21.51,18,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
2,1,55,F,112000.0,2017-07-15,2017-07,32.28,144,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
3,1,55,F,112000.0,2017-07-15,2017-07,32.28,144,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
6,2,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
7,2,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
8,2,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
9,2,75,F,100000.0,2017-05-09,2017-05,17.78,144,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
10,2,75,F,100000.0,2017-05-09,2017-05,17.78,144,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
11,2,75,F,100000.0,2017-05-09,2017-05,17.78,144,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1


In [124]:
print(f"The number of transactions in the full data: {full_data.shape[0]}")
print(f"The number of variables: {full_data.shape[1]}")

The number of transactions in the full data: 238356
The number of variables: 25


As mentioned eariler, our goal is to have a dataset that contains only viewed and completed offers where transactions made by anu customers took a place before the end of the duration of an offer. That is, the above code assure that transactions made after the end of an offer are not included in the final, clean data.

In [125]:
# showing the number of transactions by offer_type for the final and clean data

full_data['offer_type'].value_counts()

discount    139360
bogo         98996
Name: offer_type, dtype: int64

We talked about the popularity of the 10 offers we have, and we found that two discount offers were the most popular. Here, we can see than most transactions were made for discount offers. 

In [126]:
# functions to return the individual datasets with respect to each column in the full_data

def full_dataset(df, column, ev):
    data = df[df[column] == ev]
    
    return data

def offer_dataset(df, offer_num):
    offer_num = full_dataset(df, 'offer_id', offer_num)
    
    return offer_num


### <font color=blue> 2.2.1 Events

In [127]:
# creating datasets of each event for further analysis

df_received = full_dataset(full_data, 'event', 'offer received')
df_viewed = full_dataset(full_data, 'event', 'offer viewed')
df_completed = full_dataset(full_data, 'event', 'offer completed')

In [128]:
print(f"The number of received offers: {df_received.shape[0]}")

df_received.sample(3)

The number of received offers: 96626


,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
802298,7031,69,M,35000.0,2017-08-02,2017-08,1.07,108,offer received,6,...,73.418482,-1493,discount,7,168,3,1,1,1,1
956691,7558,73,F,62000.0,2016-08-28,2016-08,18.16,24,offer received,7,...,50.341957,1171,discount,20,240,5,1,0,0,1
247607,8353,82,F,81000.0,2016-05-15,2016-05,23.96,60,offer received,2,...,54.720934,-2244,bogo,10,168,10,1,1,1,0


In [130]:
# overview of the transaction where the customers did view an offer

print(f"The number of viewed offers: {df_viewed.shape[0]}")

df_viewed

The number of viewed offers: 73103


,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
7,2,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
10,2,75,F,100000.0,2017-05-09,2017-05,17.78,144,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
37,4,65,M,53000.0,2018-02-09,2018-02,9.54,60,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
39,4,65,M,53000.0,2018-02-09,2018-02,9.54,60,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
77,7,56,F,88000.0,2018-04-28,2018-04,19.91,162,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
79,7,56,F,88000.0,2018-04-28,2018-04,19.91,162,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
99,9,59,M,41000.0,2015-01-21,2015-01,0.67,60,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
151,12,40,M,33000.0,2016-07-09,2016-07,5.47,12,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
156,12,40,M,33000.0,2016-07-09,2016-07,6.18,30,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
161,12,40,M,33000.0,2016-07-09,2016-07,1.54,72,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1


In [131]:
# overview of the transaction where the customers did complete an offer

print(f"The number of completed offers: {df_completed.shape[0]}")

df_completed

The number of completed offers: 68627


,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
1,1,55,F,112000.0,2017-07-15,2017-07,21.51,18,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
3,1,55,F,112000.0,2017-07-15,2017-07,32.28,144,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
8,2,75,F,100000.0,2017-05-09,2017-05,19.89,132,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
11,2,75,F,100000.0,2017-05-09,2017-05,17.78,144,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
40,4,65,M,53000.0,2018-02-09,2018-02,9.54,60,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
53,5,57,M,42000.0,2017-12-31,2017-12,4.33,42,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
142,11,96,F,89000.0,2017-11-17,2017-11,12.03,132,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
150,12,40,M,33000.0,2016-07-09,2016-07,5.47,12,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
155,12,40,M,33000.0,2016-07-09,2016-07,6.18,30,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
160,12,40,M,33000.0,2016-07-09,2016-07,1.54,72,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1


After writting some functions that return the full data with respect to a specific variable, we will look next to top_customers customers who completed any offer and paid the most amount.

In [139]:
# making sure we have only unique_customers even if the customer made transaction more than once for an offer
unique_customers = df_completed.drop_duplicates(subset=['customer_id', 'time_of_transaction(hours)', 'offer_id'], keep='first')

# finding the top customers based on the sum of their transaction_amount of all offers completed 

top_customers = unique_customers.groupby('customer_id')['transaction_amount'].sum().sort_values(ascending=False).head(10)
top_customers

customer_id
8658     5056.42
4307     3343.89
12886    3127.56
2896     3030.39
6759     2955.24
10196    2870.32
11936    2857.77
6307     2833.60
1828     2608.13
10487    2427.42
Name: transaction_amount, dtype: float64

It looks like some customers paid thousands of dollars for Starbucks. Next, we will see the total amount of money made for each offer.

In [140]:
# finding the top offers based on the sum of the transaction_amount made by all customers who completed the offer

top_offers = unique_customers.groupby('offer_id')['transaction_amount'].sum().sort_values(ascending=False).head(10)
top_offers

offer_id
5    194632.34
7    149296.90
6    122381.83
2    110113.80
1    107063.15
4    106886.66
3     84649.89
8     77249.56
Name: transaction_amount, dtype: float64

Offers 5 and 7 have the most sum of transactions amount made by all customers.

In [141]:
# finding the average transaction_amount over all transactions and offer where customers completed the offers

df_completed['transaction_amount'].mean()

16.55074431346271

In [142]:
# creating a dataframe grouped by the time when each transaction takes a place from the start of an offer
# showing a description of the amount spent by that time

transcation_by_time = df_completed.groupby('time_of_transaction(hours)').describe()['transaction_amount'].reset_index()
transcation_by_time = transcation_by_time.drop(['std', '25%', '75%'], axis=1)
transcation_by_time

,time_of_transaction(hours),count,mean,min,50%,max
0,0,1421.0,15.365721,0.05,14.070,195.24
1,6,1854.0,16.203981,0.07,14.440,448.97
2,12,1972.0,21.033210,0.06,15.860,871.51
3,18,2127.0,20.125129,0.05,14.040,962.10
4,24,2219.0,15.679117,0.07,13.950,674.48
5,30,2430.0,15.904683,0.05,13.600,575.23
6,36,2353.0,17.465168,0.05,14.160,947.43
7,42,2485.0,16.747191,0.05,13.470,657.26
8,48,2473.0,16.036284,0.09,14.150,475.20
9,54,2391.0,16.191409,0.08,14.100,845.01


In [143]:
# a function to split the data by column and then returns a description of the all transcations grouped by their time

def transcation_by_time(df, col, target):
    transcations = df[df[col] == target]
    transcations = transcations.groupby('time_of_transaction(hours)').describe()['transaction_amount'].reset_index()
    transcations = transcations.drop(['std', '25%', '75%'], axis=1)
    return transcations

In [91]:
# plotting a trend that shows the average Amount Spent since a Start of an Offer at a specific time

f_transcations = transcation_by_time(df_completed, 'gender', 'F')
m_transcations = transcation_by_time(df_completed, 'gender', 'M')

trace_mean1 = go.Scatter(
    x=f_transcations['time_of_transaction(hours)'],
    y=f_transcations['mean'],
    name = "Female",
    line = dict(color = 'pink'),
    opacity = 0.6)

trace_mean2 = go.Scatter(
    x=m_transcations['time_of_transaction(hours)'],
    y=m_transcations['mean'],
    name = "Male",
    line = dict(color = 'cornflowerblue'),
    opacity = 0.6)

data1 = [trace_mean1, trace_mean2]

layout = {
    'title': 'Average Amount Spent Since a Start of an Offer Trend',
    'xaxis': {'title': 'Time Since a Start of an Offer (hours)'},
    'yaxis': {'title': 'Average Amount ($)', 
              "range": [
                10,
                30
            ]},
               
    'shapes': [
        # Line Horizontal, average
        {
            'type': 'line',
            'x0': 0,
            'y0': 16.55,
            'x1': 243,
            'y1': 16.55,
            'line': {
                'color': 'black',
                'width': 1,
                'dash': 'dashdot',
            }
        },
        
        # 1st highlight above average amount
        {
            'type': 'rect',
            # x-reference is assigned to the x-values
            'xref': 'paper',
            # y-reference is assigned to the plot [0,1]
            'yref': 'y',
            'x0': 0,
            'y0': 16.55,
            'x1': 1,
            'y1': 30,
            'fillcolor': 'olive',
            'opacity': 0.1,
            'line': {
                'width': 0,
            }
        },
        
        # 3nd highlight below average months
        {
            'type': 'rect',
            'xref': 'paper',
            'yref': 'y',
            'x0': 0,
            'y0': 16.55,
            'x1': 1,
            'y1': 0,
            'fillcolor': 'tomato',
            'opacity': 0.1,
            'line': {
                'width': 0,
            }
        }
    ]
}

layout.update(dict(annotations=[go.Annotation(text="Overall Average Amount ($16.55) Spent After the Start of an Offer", 
                                              x=150, 
                                              y=16.55,
                                              ax=10, 
                                              ay=-120)]))
        
fig = dict(data=data1, layout=layout)
py.iplot(fig, filename = "Amount Spent since a Start of an Offer Trend")

The trend of the transactions made since a start of an offer shows that, on average, female customers paid more than male customers at any time from the beginning of an offer up to 10 days. The only exception is transactions made after 228 hours from a start of an offer. On average, all customers paid an overall amount of $16.55 at any time, hence, we see that female customer paid more than the average at any time! Whereas male customers mostly paid less than the average.


### <font color=blue> 2.2.2 Offer Type

In [144]:
# creating datasets of each offer_type for further analysis

df_bogo = full_dataset(full_data, 'offer_type', 'bogo')
df_discount = full_dataset(full_data, 'offer_type', 'discount')

In [145]:
# overview of the transaction where the offer is BOGO

df_bogo.sample(3)

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
177928,6694,29,M,44000.0,2016-11-30,2016-11,1.06,66,offer received,2,...,54.720934,-2244,bogo,10,168,10,1,1,1,0
140420,5789,66,M,68000.0,2017-10-06,2017-10,13.00,90,offer received,2,...,54.720934,-2244,bogo,10,168,10,1,1,1,0
55125,2357,42,M,68000.0,2017-08-06,2017-08,20.09,138,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1


In [146]:
df_discount.sample(3)

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
575372,5897,20,M,39000.0,2013-08-04,2013-08,0.34,186,offer received,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
665345,7774,65,M,117000.0,2016-06-05,2016-06,27.32,84,offer received,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
474549,6779,61,M,63000.0,2015-11-09,2015-11,16.30,24,offer completed,4,...,58.980546,451,discount,10,168,2,1,1,0,1


In [147]:
# function to return top customers or offers for a specific offer type
# based on the sum of their transaction_amount of all offers completed 

def tops(df, col):
    # making sure we have only unique_customers who completed an offer even if the customer made transaction more than once for that offer
    completed_offers = df[df['event'] == 'offer completed']
    unique_customers = completed_offers.drop_duplicates(subset=['customer_id', 'time_of_transaction(hours)', 'offer_id'], keep='first')

    # finding the tops based on the sum of their transaction_amount of all offers completed for that specific offer type 

    top1 = unique_customers.groupby(col)['transaction_amount'].sum().sort_values(ascending=False).head(10)
    
    return top1

def tops_by_gender(df):
    # making sure we have only unique_customers who completed an offer even if the customer made transaction more than once for that offer
    completed_offers = df[df['event'] == 'offer completed']
    unique_customers = completed_offers.drop_duplicates(subset=['customer_id', 'time_of_transaction(hours)', 'offer_id'], keep='first')

    # finding the amount of all completed offer by gender

    amount = unique_customers.groupby(['offer_id', 'gender'])['transaction_amount'].sum()
    
    return amount

def customer_report(cid, df=df_discount):
    report = df[df.event == 'offer completed']
    report = report[report.customer_id == cid]
    
    return report

In [148]:
# finding the top customers based on the sum of their transaction_amount of all BOGO offers completed 

tops(df_bogo, 'customer_id')

customer_id
4307     1990.88
1828     1724.50
261      1573.74
10822    1515.80
2896     1507.78
621      1465.94
6307     1416.80
10080    1408.48
6624     1301.02
10487    1184.38
Name: transaction_amount, dtype: float64

As we now focus on transactions made for only the BOGO offers, we can see the top 10 customers who paid the most for all BOGO offers

In [149]:
# finding the top customers based on the sum of their transaction_amount of all discount offers completed 

tops(df_discount, 'customer_id')

customer_id
8658     4057.18
12886    3127.56
6759     2216.43
9904     2049.56
12108    2019.64
10196    1933.38
11936    1928.68
10048    1853.52
11791    1765.39
7923     1740.66
Name: transaction_amount, dtype: float64

Here, we find the top 10 customers with respect to the discount offers.

In [150]:
# reporting more info about a customer

customer_report(8658, df_discount)

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
537915,8658,60,M,106000.0,2016-07-02,2016-07,24.23,30,offer completed,4,...,58.980546,451,discount,10,168,2,1,1,0,1
537918,8658,60,M,106000.0,2016-07-02,2016-07,947.43,36,offer completed,4,...,58.980546,451,discount,10,168,2,1,1,0,1
537921,8658,60,M,106000.0,2016-07-02,2016-07,27.58,120,offer completed,4,...,58.980546,451,discount,10,168,2,1,1,0,1
707286,8658,60,M,106000.0,2016-07-02,2016-07,24.23,30,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
707289,8658,60,M,106000.0,2016-07-02,2016-07,947.43,36,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
707292,8658,60,M,106000.0,2016-07-02,2016-07,27.58,120,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
707295,8658,60,M,106000.0,2016-07-02,2016-07,30.11,180,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
877717,8658,60,M,106000.0,2016-07-02,2016-07,24.23,30,offer completed,6,...,73.418482,-1493,discount,7,168,3,1,1,1,1
877720,8658,60,M,106000.0,2016-07-02,2016-07,947.43,36,offer completed,6,...,73.418482,-1493,discount,7,168,3,1,1,1,1
877723,8658,60,M,106000.0,2016-07-02,2016-07,27.58,120,offer completed,6,...,73.418482,-1493,discount,7,168,3,1,1,1,1


In [151]:
# finding the top offers based on the sum of the transaction_amount made by all customers who completed the offer

tops(df_bogo, 'offer_id')

offer_id
2    110113.80
1    107063.15
3     84649.89
8     77249.56
Name: transaction_amount, dtype: float64

As mentioned eariler, we have 4 BOGO offers and 4 Discount offers. Here we see that offer 2 had the most total transactions amount over the other BOGO offers.

In [152]:
# finding the top offers based on the sum of the transaction_amount made by all customers who completed the offer

tops(df_discount, 'offer_id')

offer_id
5    194632.34
7    149296.90
6    122381.83
4    106886.66
Name: transaction_amount, dtype: float64

Offer 5 made the most among other Discount offers, and overall!

In [153]:
# finding the total amount for each BOGO offer by gender 

tops_by_gender(df_bogo)

offer_id  gender
1         F         55421.53
          M         49555.68
          O          2085.94
2         F         56392.20
          M         51419.22
          O          2302.38
3         F         43190.65
          M         40197.47
          O          1261.77
8         F         42923.16
          M         33526.68
          O           799.72
Name: transaction_amount, dtype: float64

More details about the total transactions amount made for each BOGO offer by customers with respect to their gender. Overall, female customers paid more than male customers. Both, Female and male customers paid most for offer 2.

In [154]:
# finding the total amount for each discount offer by gender 

tops_by_gender(df_discount)

offer_id  gender
4         F         56113.92
          M         48823.54
          O          1949.20
5         F         98573.24
          M         93458.66
          O          2600.44
6         F         63224.15
          M         57367.13
          O          1790.55
7         F         76970.11
          M         69990.11
          O          2336.68
Name: transaction_amount, dtype: float64

With respect to Discount offers, again, femal customers paid more than male customers. Both paid the most for offer 5.


### <font color=blue> 2.2.3 Gender

In [155]:
# creating datasets for each gender for further analysis

df_male = full_dataset(full_data, 'gender', 'M')
df_female = full_dataset(full_data, 'gender', 'F')
df_other = full_dataset(full_data, 'gender', 'O')

In [156]:
# overview of the transactions made by male customers
df_male.sample(3)

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
573132,12835,18,M,40000.0,2016-04-22,2016-04,9.13,162,offer received,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
680082,13350,70,M,59000.0,2016-08-27,2016-08,16.58,198,offer viewed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
331532,9891,74,M,53000.0,2015-09-22,2015-09,1.25,78,offer viewed,3,...,62.393552,-2207,bogo,5,120,5,1,1,1,1


In [157]:
# overview of the transactions made by female customers

df_female.sample(3)

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
357102,2958,65,F,51000.0,2016-03-25,2016-03,22.93,108,offer viewed,3,...,62.393552,-2207,bogo,5,120,5,1,1,1,1
715034,5087,29,F,60000.0,2017-09-24,2017-09,11.05,192,offer received,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
602910,1574,87,F,83000.0,2017-11-06,2017-11,21.17,150,offer viewed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1


In [158]:
# finding the top male customers based on the sum of their transaction_amount

tops(df_male, 'customer_id')

customer_id
8658     5056.42
4307     3343.89
6307     2833.60
261      2360.61
10048    2316.90
10822    2273.70
8260     2107.81
7862     2030.46
8976     1751.50
7923     1740.66
Name: transaction_amount, dtype: float64

Eariler, we looked at the overall top customers. Here, we find the top 10 male customers based on the total amount they paid for all offer.

In [159]:
# finding the top female customers based on the sum of their transaction_amount

tops(df_female, 'customer_id')

customer_id
12886    3127.56
2896     3030.39
6759     2955.24
10196    2870.32
11936    2857.77
1828     2608.13
10487    2427.42
11791    2344.68
3342     2166.70
9904     2124.15
Name: transaction_amount, dtype: float64

The top 10 female customers.

In [160]:
# reporting more info about a customer

customer_report(9904, df_female)

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
332417,9904,58,F,64000.0,2015-11-16,2015-11,14.43,6,offer completed,3,...,62.393552,-2207,bogo,5,120,5,1,1,1,1
332420,9904,58,F,64000.0,2015-11-16,2015-11,25.08,30,offer completed,3,...,62.393552,-2207,bogo,5,120,5,1,1,1,1
332423,9904,58,F,64000.0,2015-11-16,2015-11,21.71,48,offer completed,3,...,62.393552,-2207,bogo,5,120,5,1,1,1,1
332426,9904,58,F,64000.0,2015-11-16,2015-11,13.37,108,offer completed,3,...,62.393552,-2207,bogo,5,120,5,1,1,1,1
615311,9904,58,F,64000.0,2015-11-16,2015-11,14.43,6,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
615314,9904,58,F,64000.0,2015-11-16,2015-11,25.08,30,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
615317,9904,58,F,64000.0,2015-11-16,2015-11,21.71,48,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
615320,9904,58,F,64000.0,2015-11-16,2015-11,13.37,108,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
615323,9904,58,F,64000.0,2015-11-16,2015-11,11.20,132,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1
615326,9904,58,F,64000.0,2015-11-16,2015-11,570.78,138,offer completed,5,...,75.210463,-1404,discount,10,240,2,1,1,1,1


In [161]:
# finding the top offers based on the sum of the transaction_amount made by male customers who completed that offer

tops(df_male, 'offer_id')

offer_id
5    93458.66
7    69990.11
6    57367.13
2    51419.22
1    49555.68
4    48823.54
3    40197.47
8    33526.68
Name: transaction_amount, dtype: float64

In [162]:
# finding the top offers based on the sum of the transaction_amount made by male customers who completed that offer

tops(df_female, 'offer_id')

offer_id
5    98573.24
7    76970.11
6    63224.15
2    56392.20
4    56113.92
1    55421.53
3    43190.65
8    42923.16
Name: transaction_amount, dtype: float64

In [164]:
# function to return plots with respect to different distributions by gender

def age_distributions(df, df2, color, color2):
    df = df.drop_duplicates(subset='customer_id', keep='first')
    df2 = df2.drop_duplicates(subset='customer_id', keep='first')
    x = df.age
    x2 = df2.age

    trace1 = go.Histogram(
        x=x,
        name='Female',
        opacity=0.6,
        nbinsx = 7,
        marker=dict(
            color=color)
    )
    
    trace2 = go.Histogram(
        x=x2,
        name='Male',
        opacity=0.6,
        nbinsx = 7,
        marker=dict(
            color=color2)
    )

    data1 = [trace1, trace2]
    layout = go.Layout(
        barmode='stack',
        bargap=0.1,
        title = 'Age by Gender',
        xaxis=dict(
            title='Age'),
        yaxis=dict(
            title='Total number of Customers'))
    
    updatemenus = list([
    dict(active=0,
         buttons=list([   
            dict(label = 'All',
                 method = 'update',
                 args = [{'visible': [True, True]},
                         {'title': 'Age Distribution by Gender'}]),
             dict(label = 'Female',
                 method = 'update',
                 args = [{'visible': [True, False]},
                         {'title': 'Age Distribution of Male Customers'}]),
            dict(label = 'Male',
                 method = 'update',
                 args = [{'visible': [False, True]},
                         {'title': 'Age Distribution of Female Customers'}])
            ]),
        )
    ])

    layout.update(dict(updatemenus=updatemenus))

    fig = go.Figure(data=data1, layout=layout)
    plot = py.iplot(fig, filename='Age by Gender')
    
    return plot

def income_distributions(df, df2, color, color2):
    df = df.drop_duplicates(subset='customer_id', keep='first')
    df2 = df2.drop_duplicates(subset='customer_id', keep='first')
    x = df.income
    x2 = df2.income

    trace1 = go.Histogram(
        x=x,
        name='Female',
        opacity=0.6,
        nbinsx = 7,
        marker=dict(
            color=color)
    )
    
    trace2 = go.Histogram(
        x=x2,
        name='Male',
        opacity=0.6,
        nbinsx = 7,
        marker=dict(
            color=color2)
    )

    data2 = [trace1, trace2]
    layout = go.Layout(
        barmode='stack',
        bargap=0.1,
        title = 'Income by Gender',
        xaxis=dict(
            title='Income'),
        yaxis=dict(
            title='Total number of Customers'))
    
        
    updatemenus = list([
    dict(active=0,
         buttons=list([
            
            dict(label = 'All',
                 method = 'update',
                 args = [{'visible': [True, True]},
                         {'title': 'Income Distribution by Gender'}]),

            dict(label = 'Female',
                 method = 'update',
                 args = [{'visible': [True, False]},
                         {'title': 'Income Distribution of Male Customers'}]),
            dict(label = 'Male',
                 method = 'update',
                 args = [{'visible': [False, True]},
                         {'title': 'Income Distribution of Female Customers'}])

            ]),
        )
    ])

    layout.update(dict(updatemenus=updatemenus))

    fig = go.Figure(data=data2, layout=layout)
    plot = py.iplot(fig, filename='Income by Gender')
    
    return plot

# function to return an analysis of the numbers and percentage of all transaction by each event

def analysis(df1, col1, col2, col3):
    v = df1[df1['event'] == 'offer viewed']
    v = v.drop_duplicates(subset=['customer_id', 'offer_id'], keep='first')

    c = df1[df1['event'] == 'offer completed']
    c = c.drop_duplicates(subset=['customer_id', 'offer_id'], keep='first')

    received = df1.drop_duplicates(subset=['customer_id', 'offer_id'], keep='first')

    viewed = pd.Series(v.offer_id).value_counts().reset_index().rename(columns={'index': 'offer_id_v', 'offer_id': 'viewed'})
    completed = pd.Series(c.offer_id).value_counts().reset_index().rename(columns={'index': 'offer_id_c', 'offer_id': 'completed'})
    received = pd.Series(received.offer_id).value_counts().reset_index().rename(columns={'index': 'offer_id_r', 'offer_id': 'received'})


    analysis = pd.concat([received, viewed, completed], axis=1).rename(columns={'offer_id_r': 'offer_id', 'viewed': col2, 'completed': col3, 'received': col1})
    analysis[col2+'(%)'] = round(analysis[col2]/analysis[col1]*100, 2)
    analysis[col3+'(%)'] = round(analysis[col3]/analysis[col1]*100, 2)

    analysis = analysis.drop(['offer_id_c', 'offer_id_v'], axis=1)
    
    return analysis

In [115]:
# plotting the age_distributions 

age_distributions(df_female, df_male, 'pink', 'cornflowerblue')

A plot shows that most customers age between 40 and 60 years old.

In [118]:
# plotting the income_distributions 

income_distributions(df_female, df_male, 'pink', 'cornflowerblue')

Here, we confirm the previous finding about the income distributions. The plot confirms that most customers make between 40K and 60K. Most of the female customers make between 60K and 80K while most male customer make between 40K and 60K a year.

In [165]:
# showing an analysis of each offer for both genders; male and female customers

overall_analysis = analysis(full_data, 'received', 'overall_views', 'overall_completion')
overall_analysis

,offer_id,received,overall_views,overall_completion,overall_views(%),overall_completion(%)
0,5,4533,4402,3729,97.11,82.26
1,7,4451,3711,3169,83.37,71.20
2,2,3904,3552,2773,90.98,71.03
3,6,3837,3276,2592,85.38,67.55
4,1,3802,3248,2573,85.43,67.67
5,4,3776,2175,2499,57.60,66.18
6,3,3385,2132,2374,62.98,70.13
7,8,3349,1607,1917,47.98,57.24


In [166]:
# showing an analysis of each offer for male customers

male_analysis = analysis(df_male, 'received', 'male_views', 'male_completion')
male_analysis

,offer_id,received,male_views,male_completion,male_views(%),male_completion(%)
0,5,2639,2554,2052,96.78,77.76
1,7,2554,2174,1755,85.12,68.72
2,2,2275,2106,1441,92.57,63.34
3,6,2250,1884,1354,83.73,60.18
4,1,2194,1878,1324,85.60,60.35
5,4,2166,1196,1265,55.22,58.40
6,8,1944,1156,1143,59.47,58.80
7,3,1941,861,917,44.36,47.24


In [167]:
# showing an analysis of each offer for female customers

female_analysis = analysis(df_female, 'received', 'female_views', 'female_completion')
female_analysis

,offer_id,received,female_views,female_completion,female_views(%),female_completion(%)
0,5,1836,1790,1626,97.49,88.56
1,7,1835,1483,1371,80.82,74.71
2,2,1569,1393,1281,88.78,81.64
3,4,1556,1353,1261,86.95,81.04
4,1,1541,1324,1197,85.92,77.68
5,6,1531,934,1188,61.01,77.60
6,3,1396,929,1134,66.55,81.23
7,8,1365,711,970,52.09,71.06


In [168]:
# preparing a dataframe shows the completion percentages of each offer by gender

completion_perc = pd.merge(pd.merge(female_analysis, 
                                    male_analysis, on='offer_id'),
                           overall_analysis, on='offer_id')
col_list = ['offer_id', 'female_completion(%)', 'male_completion(%)', 'overall_completion(%)']
completion_perc = completion_perc[col_list].sort_values(by='offer_id').set_index('offer_id')

completion_perc

,female_completion(%),male_completion(%),overall_completion(%)
offer_id,,,
1,77.68,60.35,67.67
2,81.64,63.34,71.03
3,81.23,47.24,70.13
4,81.04,58.40,66.18
5,88.56,77.76,82.26
6,77.60,60.18,67.55
7,74.71,68.72,71.20
8,71.06,58.80,57.24


After the data has been cleaned even further during the previous processes, here is a completion rate comparison of the genders for each offer. The overall completion rate account for customers, again, offer 5 is the most popular offer. Oviously, female customers reponse to offer more than male customers. The following chart illustrates the number where we can see a clear difference between female customers actions toward offers and the male customers response.

In [121]:
# plotting the Percentages of Completed offers by Gender

f_completed = completion_perc['female_completion(%)']
m_completed = completion_perc['male_completion(%)']
overall_completed = completion_perc['overall_completion(%)']
offers = completion_perc.index

x = offers
y = f_completed
y2 = m_completed
y3 = overall_completed

trace1 = go.Bar(
    x=x,
    y=y,
    name='Female',
    #hoverinfo = 'y',
    hovertemplate = '<i>Percentage of all Female Customers who completed the Offer</i>: %{y:.2f}%'
                    '<br><b>Offer</b>: %{x}<br>',
    marker=dict(
        color='pink',
        line=dict(
            color='grey',
            width=1.5),
        ),
    opacity=0.6
)

trace2 = go.Bar(
    x=x,
    y=y2,
    name='Male',
    #hoverinfo = 'y',
    hovertemplate = '<i>Percentage of all Male Customers who completed the Offer</i>: %{y:.2f}%'
                    '<br><b>Offer</b>: %{x}<br>',
    marker=dict(
        color='cornflowerblue',
        line=dict(
            color='grey',
            width=1.5),
        ),
    opacity=0.6
)



trace3 = go.Scatter(
    x=x,
    y=y3,
    name='Overall',
    #hoverinfo= 'y',
    hovertemplate = '<i>Percentage of all Customers, Male, Female, and Other who completed the Offer</i>: %{y:.2f}%'
                    '<br><b>Offer</b>: %{x}<br>',
    marker=dict(
        color='grey',
        )
)

data1 = [trace1, trace2, trace3]

layout = go.Layout(
    title = "Percentage of Completed offers by Gender",
    xaxis=dict(title = 'Offers',
    type='category'),
    barmode='group',
    yaxis = dict(title = 'Percentage of Completed offers'
        #hoverformat = '.2f'
    )
)

fig = go.Figure(data=data1, layout=layout)
py.iplot(fig, filename='Percentage of Completed offers by Gender')

/Users/Saleh/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead




### <font color=blue> 2.2.4 Offers

In [248]:
offers_list = full_data.drop_duplicates(subset=['offer_id'], keep='first')
offers_list = offers_list.drop(['customer_id', 
                                'age', 
                                'gender', 
                                'membership_start', 'membership_period', 
                                'transaction_amount', 'time_of_transaction(hours)',
                                'event', 
                                'income', 
                                'time(hours)_y',
                                'offer_views(%)',
                                'completed(not_viewed)'], axis=1)
offers_list.set_index('offer_id')

,received,viewed,completed,offer_completion(%),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
offer_id,,,,,,,,,,,,
1,6685,3499,4188,62.647719,bogo,5,168,5,1,1,0,1
2,6683,5901,3657,54.720934,bogo,10,168,10,1,1,1,0
3,6576,6310,4103,62.393552,bogo,5,120,5,1,1,1,1
4,6631,3460,3911,58.980546,discount,10,168,2,1,1,0,1
5,6652,6407,5003,75.210463,discount,10,240,2,1,1,1,1
6,6655,6379,4886,73.418482,discount,7,168,3,1,1,1,1
7,6726,2215,3386,50.341957,discount,20,240,5,1,0,0,1
8,6593,6329,3310,50.204763,bogo,10,120,10,1,1,1,1


In [246]:
offers_list[offers_list.offer_type == 'bogo'].set_index('offer_id')

,received,viewed,completed,offer_completion(%),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
offer_id,,,,,,,,,,,,
1,6685,3499,4188,62.647719,bogo,5,168,5,1,1,0,1
2,6683,5901,3657,54.720934,bogo,10,168,10,1,1,1,0
3,6576,6310,4103,62.393552,bogo,5,120,5,1,1,1,1
8,6593,6329,3310,50.204763,bogo,10,120,10,1,1,1,1


In [247]:
offers_list[offers_list.offer_type == 'discount'].set_index('offer_id')

,received,viewed,completed,offer_completion(%),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
offer_id,,,,,,,,,,,,
4,6631,3460,3911,58.980546,discount,10,168,2,1,1,0,1
5,6652,6407,5003,75.210463,discount,10,240,2,1,1,1,1
6,6655,6379,4886,73.418482,discount,7,168,3,1,1,1,1
7,6726,2215,3386,50.341957,discount,20,240,5,1,0,0,1


In [169]:
# function to return an analysis of the numbers and percentage of all transaction by offer
def analysis2(df1, col1, col2, col3, offer):
    v = df1[df1['event'] == 'offer viewed']
    v = v.drop_duplicates(subset=['customer_id', 'offer_id'], keep='first')

    c = df1[df1['event'] == 'offer completed']
    c = c.drop_duplicates(subset=['customer_id', 'offer_id'], keep='first')

    received = df1.drop_duplicates(subset=['customer_id', 'offer_id'], keep='first')

    viewed = pd.Series(v.age).value_counts().reset_index().rename(columns={'index': 'age_v', 'age': 'viewed'})
    completed = pd.Series(c.age).value_counts().reset_index().rename(columns={'index': 'age_c', 'age': 'completed'})
    received = pd.Series(received.age).value_counts().reset_index().rename(columns={'index': 'age_r', 'age': 'received'})


    analysis2 = pd.concat([viewed, completed, received], axis=1).rename(columns={'age_v': 'age', 'viewed': col2, 'completed': col3, 'received': col1})
    analysis2['offer'+offer+'_'+col2+'(%)'] = round(analysis2[col2]/analysis2[col1]*100, 2)
    analysis2['offer'+offer+'_'+col3+'(%)'] = round(analysis2[col3]/analysis2[col1]*100, 2)

    analysis2 = analysis2.drop(['age_r', 'age_c'], axis=1).sort_values(by='age')
    
    return analysis2

# function to return dataframe grouped by the time when each transaction takes a place from the start of an offer
# showing a description of the amount spent by that time
def offer_transcations(offer):
    offer_trans = transcation_by_time(df_completed, 'offer_id', offer)
    
    return offer_trans

# function to plot a trend of any two offers that shows the average Amount Spent since a Start of an Offer at a specific time
def plot2(offer1, offer2):
    offer_trans1 = offer_transcations(offer1)
    offer_trans2 = offer_transcations(offer2)
    
    trace_mean1 = go.Scatter(
        x=offer_trans1['time_of_transaction(hours)'],
        y=offer_trans1['mean'],
        name = offer1,
        opacity = 0.6)
    trace_mean2 = go.Scatter(
        x=offer_trans2['time_of_transaction(hours)'],
        y=offer_trans2['mean'],
        name = offer2,
        opacity = 0.6)


    data1 = [trace_mean1, trace_mean2]

    layout = {
        'title': 'Average Amount Spent Since a Start of an Offer Trend',
        'xaxis': {'title': 'Time Since a Start of an Offer (hours)'},
        'yaxis': {'title': 'Average Amount ($)', 
                  "range": [
                    10,
                    35
                ]}}
        
    fig = dict(data=data1, layout=layout)
    plot2 = py.iplot(fig, filename = "Amount Spent since a Start of an Offer Trend by Offer")
    
    return plot2

In [170]:
# showing a description of the amount spent by the time of transactions

offer_transcations(1)

,time_of_transaction(hours),count,mean,min,50%,max
0,0,176.0,16.762386,0.16,15.185,195.24
1,6,190.0,18.077684,0.07,14.780,448.97
2,12,260.0,18.776923,0.21,15.565,639.59
3,18,252.0,15.691270,0.13,13.245,366.65
4,24,261.0,15.152414,0.07,13.770,42.30
5,30,308.0,15.146201,0.12,12.870,323.88
6,36,289.0,16.060796,0.05,13.990,400.74
7,42,325.0,18.911662,0.18,13.100,657.26
8,48,310.0,17.156097,0.09,14.725,475.20
9,54,267.0,14.922097,0.43,12.740,347.37


In [171]:
# plotting and comparing the trend of any two offers

plot2(5, 7)

/Users/Saleh/anaconda3/lib/python3.7/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



The plot shows that the average amount spent on offer 5 is higher than what was spent on offer 7 during any time after both offers have started at.

In [172]:
# creating a dataframe that contains only transactions by a specific offer

offer1 = offer_dataset(full_data, 1)
offer1.sample(3)

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
52818,2247,38,F,55000.0,2016-06-24,2016-06,9.62,60,offer viewed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
5871,258,63,F,74000.0,2016-07-28,2016-07,32.05,66,offer received,1,...,62.647719,689,bogo,5,168,5,1,1,0,1
34196,1487,43,F,58000.0,2015-10-07,2015-10,18.43,84,offer completed,1,...,62.647719,689,bogo,5,168,5,1,1,0,1


In [173]:
# showing the numbers and percentages of all transactions made by a specific age for a specific offer

offer1_analysis = analysis2(offer1, 'received', 'viewed', 'completed', '1')
offer1_analysis

,age,viewed,completed,received,offer1_viewed(%),offer1_completed(%)
68,18.0,9.0,12.0,16,56.25,75.00
60,19.0,13.0,19.0,30,43.33,63.33
56,20.0,15.0,22.0,33,45.45,66.67
57,21.0,14.0,21.0,31,45.16,67.74
58,22.0,14.0,20.0,30,46.67,66.67
61,23.0,13.0,17.0,28,46.43,60.71
52,24.0,18.0,25.0,37,48.65,67.57
62,25.0,12.0,17.0,28,42.86,60.71
49,26.0,19.0,27.0,38,50.00,71.05
42,27.0,21.0,29.0,42,50.00,69.05


In [174]:
offer2 = offer_dataset(full_data, 2)
offer2.sample(3)

,customer_id,age,gender,income,membership_start,membership_period,transaction_amount,time_of_transaction(hours),event,offer_id,...,offer_completion(%),completed(not_viewed),offer_type,difficulty($),duration(hours),reward,email,mobile,social,web
164579,6375,75,F,31000.0,2015-11-10,2015-11,9.88,54,offer viewed,2,...,54.720934,-2244,bogo,10,168,10,1,1,1,0
201992,7267,69,M,89000.0,2016-09-24,2016-09,26.76,48,offer viewed,2,...,54.720934,-2244,bogo,10,168,10,1,1,1,0
211818,7509,25,F,32000.0,2014-11-23,2014-11,0.14,144,offer viewed,2,...,54.720934,-2244,bogo,10,168,10,1,1,1,0


In [175]:
offer2_analysis = analysis2(offer2, 'received', 'viewed', 'completed', '2')
offer2_analysis

,age,viewed,completed,received,offer2_viewed(%),offer2_completed(%)
69,18,16,8,18,88.89,44.44
57,19,32,17,33,96.97,51.52
42,20,42,25,44,95.45,56.82
53,21,33,18,34,97.06,52.94
55,22,33,17,33,100.00,51.52
49,23,35,19,39,89.74,48.72
47,24,38,20,41,92.68,48.78
39,25,46,27,47,97.87,57.45
40,26,45,26,47,95.74,55.32
54,27,33,18,33,100.00,54.55


In [176]:
offer3 = offer_dataset(full_data, 3)
offer3_analysis = analysis2(offer3, 'received', 'viewed', 'completed', '3')
offer3_analysis

,age,viewed,completed,received,offer3_viewed(%),offer3_completed(%)
68,18,17,13,17,100.00,76.47
41,19,37,27,38,97.37,71.05
44,20,35,25,36,97.22,69.44
40,21,37,27,38,97.37,71.05
55,22,29,21,30,96.67,70.00
43,23,36,25,38,94.74,65.79
52,24,32,22,33,96.97,66.67
47,25,33,23,35,94.29,65.71
28,26,46,34,47,97.87,72.34
45,27,35,25,36,97.22,69.44


In [177]:
offer4 = offer_dataset(full_data, 4)
offer4_analysis = analysis2(offer4, 'received', 'viewed', 'completed', '4')
offer4_analysis

,age,viewed,completed,received,offer4_viewed(%),offer4_completed(%)
71,18.0,6.0,9,12,50.00,75.00
64,19.0,11.0,13,23,47.83,56.52
54,20.0,15.0,20,33,45.45,60.61
50,21.0,16.0,21,37,43.24,56.76
59,22.0,13.0,17,29,44.83,58.62
49,23.0,16.0,22,38,42.11,57.89
44,24.0,18.0,24,42,42.86,57.14
60,25.0,12.0,17,28,42.86,60.71
58,26.0,13.0,17,29,44.83,58.62
57,27.0,13.0,17,29,44.83,58.62


In [178]:
offer5 = offer_dataset(full_data, 5)
offer5_analysis = analysis2(offer5, 'received', 'viewed', 'completed', '5')
offer5_analysis

,age,viewed,completed,received,offer5_viewed(%),offer5_completed(%)
66,18,21,19,22,95.45,86.36
53,19,40,31,41,97.56,75.61
61,20,31,25,32,96.88,78.12
41,21,49,40,53,92.45,75.47
49,22,42,33,44,95.45,75.00
48,23,42,34,44,95.45,77.27
55,24,38,30,40,95.00,75.00
44,25,46,38,49,93.88,77.55
39,26,54,43,56,96.43,76.79
40,27,53,43,55,96.36,78.18


In [179]:
offer6 = offer_dataset(full_data, 6)
offer6_analysis = analysis2(offer6, 'received', 'viewed', 'completed', '6')
offer6_analysis

,age,viewed,completed,received,offer6_viewed(%),offer6_completed(%)
66,18,21,19.0,23,91.30,82.61
45,19,43,34.0,44,97.73,77.27
56,20,33,25.0,33,100.00,75.76
47,21,40,33.0,41,97.56,80.49
50,22,38,31.0,39,97.44,79.49
51,23,37,29.0,38,97.37,76.32
48,24,39,32.0,41,95.12,78.05
52,25,36,28.0,38,94.74,73.68
43,26,44,36.0,45,97.78,80.00
38,27,46,39.0,47,97.87,82.98


In [180]:
offer7 = offer_dataset(full_data, 7)
offer7_analysis = analysis2(offer7, 'received', 'viewed', 'completed', '7')
offer7_analysis

,age,viewed,completed,received,offer7_viewed(%),offer7_completed(%)
68,18.0,6.0,11,19,31.58,57.89
72,19.0,5.0,8,12,41.67,66.67
58,20.0,7.0,17,34,20.59,50.00
65,21.0,6.0,12,26,23.08,46.15
66,22.0,6.0,11,24,25.00,45.83
45,23.0,11.0,23,47,23.40,48.94
44,24.0,12.0,24,48,25.00,50.00
50,25.0,10.0,20,44,22.73,45.45
62,26.0,7.0,13,31,22.58,41.94
67,27.0,6.0,11,22,27.27,50.00


In [181]:
offer8 = offer_dataset(full_data, 8)
offer8_analysis = analysis2(offer8, 'received', 'viewed', 'completed', '8')
offer8_analysis

,age,viewed,completed,received,offer8_viewed(%),offer8_completed(%)
64,18,20,10.0,20,100.00,50.00
41,19,38,18.0,39,97.44,46.15
55,20,29,14.0,30,96.67,46.67
38,21,39,22.0,40,97.50,55.00
57,22,28,13.0,29,96.55,44.83
40,23,38,19.0,40,95.00,47.50
45,24,35,16.0,38,92.11,42.11
46,25,35,16.0,37,94.59,43.24
39,26,38,22.0,40,95.00,55.00
32,27,42,26.0,42,100.00,61.90


In [182]:
# merging all the analyses created for each offer and then create a dataframe shows the completion percentage 
# of each offer based on the age group

completion_perc_o = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(offer1_analysis, offer2_analysis, on='age'), 
                             offer3_analysis, on='age'),
                             offer4_analysis, on='age'),
                             offer5_analysis, on='age'),
                             offer6_analysis, on='age'),
                             offer7_analysis, on='age'),
                             offer8_analysis, on='age')
col_list = ['age', 
            'offer1_completed(%)', 
            'offer2_completed(%)', 
            'offer3_completed(%)', 
            'offer4_completed(%)', 
            'offer5_completed(%)', 
            'offer6_completed(%)', 
            'offer7_completed(%)', 
            'offer8_completed(%)']
completion_perc_o = completion_perc_o[col_list]
completion_perc_o

,age,offer1_completed(%),offer2_completed(%),offer3_completed(%),offer4_completed(%),offer5_completed(%),offer6_completed(%),offer7_completed(%),offer8_completed(%)
0,18.0,75.00,44.44,76.47,75.00,86.36,82.61,57.89,50.00
1,19.0,63.33,51.52,71.05,56.52,75.61,77.27,66.67,46.15
2,20.0,66.67,56.82,69.44,60.61,78.12,75.76,50.00,46.67
3,21.0,67.74,52.94,71.05,56.76,75.47,80.49,46.15,55.00
4,22.0,66.67,51.52,70.00,58.62,75.00,79.49,45.83,44.83
5,23.0,60.71,48.72,65.79,57.89,77.27,76.32,48.94,47.50
6,24.0,67.57,48.78,66.67,57.14,75.00,78.05,50.00,42.11
7,25.0,60.71,57.45,65.71,60.71,77.55,73.68,45.45,43.24
8,26.0,71.05,55.32,72.34,58.62,76.79,80.00,41.94,55.00
9,27.0,69.05,54.55,69.44,58.62,78.18,82.98,50.00,61.90


A complete analysis of each offer was created to show the completion rate of each age group with respect to an offer. The following function will allow us to report the completion rates per age group. Then, we will plot a trend of the completion rates by each age group for each offer.

In [183]:
# a function to return a report of an age group that contains all completion percentages by offer

def age_report(a, df=completion_perc_o):
    report = df[df.age == a]
    
    return report

In [184]:
age_report(35)

,age,offer1_completed(%),offer2_completed(%),offer3_completed(%),offer4_completed(%),offer5_completed(%),offer6_completed(%),offer7_completed(%),offer8_completed(%)
17,35.0,67.57,56.82,64.29,68.42,72.5,82.61,51.35,44.83


In [185]:
age_report(40)

,age,offer1_completed(%),offer2_completed(%),offer3_completed(%),offer4_completed(%),offer5_completed(%),offer6_completed(%),offer7_completed(%),offer8_completed(%)
22,40.0,73.68,63.24,78.85,67.27,81.82,84.21,56.96,64.29


In [186]:
age_report(25)

,age,offer1_completed(%),offer2_completed(%),offer3_completed(%),offer4_completed(%),offer5_completed(%),offer6_completed(%),offer7_completed(%),offer8_completed(%)
7,25.0,60.71,57.45,65.71,60.71,77.55,73.68,45.45,43.24


In [229]:
# creating a copy of the completion reports 

plot1 = completion_perc_o.copy()

# plotting the Percentages of Completed offers by each Age group for each offer

plot1 = plot1.set_index('age')

trace1 = go.Scatter(
    x=plot1.index,
    y=plot1['offer1_completed(%)'],
    name = "Offer 1",
    opacity = 0.8)

trace2 = go.Scatter(
    x=plot1.index,
    y=plot1['offer2_completed(%)'],
    name = "Offer 2",
    opacity = 0.8)

trace3 = go.Scatter(
    x=plot1.index,
    y=plot1['offer3_completed(%)'],
    name = "Offer 3",
    opacity = 0.8)

trace4 = go.Scatter(
    x=plot1.index,
    y=plot1['offer4_completed(%)'],
    name = "Offer 4",
    opacity = 0.8)

trace5 = go.Scatter(
    x=plot1.index,
    y=plot1['offer5_completed(%)'],
    name = "Offer 5",
    opacity = 0.8)

trace6 = go.Scatter(
    x=plot1.index,
    y=plot1['offer6_completed(%)'],
    name = "Offer 6",
    opacity = 0.8)

trace7 = go.Scatter(
    x=plot1.index,
    y=plot1['offer7_completed(%)'],
    name = "Offer 7",
    opacity = 0.8)

trace8 = go.Scatter(
    x=plot1.index,
    y=plot1['offer8_completed(%)'],
    name = "Offer 8",
    opacity = 0.8)

data1 = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8]

layout = {
    'title': 'Percentage of Completed offers by Age',
    'xaxis': {'title': 'Age'},
    'yaxis': {'title': 'Percentage Completed (%)'}}

layout.update(dict(xaxis=dict(rangeslider=dict(visible = True),type='linear')))

updatemenus = list([
    dict(active=0,
         buttons=list([   
            dict(label = 'All',
                 method = 'update',
                 args = [{'visible': [True, True, True, True, True, True, True, True]},
                         {'title': 'Percentage of Each Completed offers by Age'}]),
            dict(label = 'Offer 1',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, False, False, False, False]},
                         {'title': 'Percentage of Completed offers by Age for Offer 1'}]),
            dict(label = 'Offer 2',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, False, False, False, False]},
                         {'title': 'Percentage of Completed offers by Age for Offer 2'}]),
            dict(label = 'Offer 3',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, False, False, False, False]},
                         {'title': 'Percentage of Completed offers by Age for Offer 3'}]),
            dict(label = 'Offer 4',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, False, False, False, False]},
                         {'title': 'Percentage of Completed offers by Age for Offer 4'}]),
            dict(label = 'Offer 5',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, True, False, False, False]},
                         {'title': 'Percentage of Completed offers by Age for Offer 5'}]),
            dict(label = 'Offer 6',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, True, False, False]},
                         {'title': 'Percentage of Completed offers by Age for Offer 6'}]),
            dict(label = 'Offer 7',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, True, False]},
                         {'title': 'Percentage of Completed offers by Age for Offer 7'}]),
            dict(label = 'Offer 8',
                 method = 'update',
                 args = [{'visible': [False, False, False, False, False, False, False, True]},
                         {'title': 'Percentage of Completed offers by Age for Offer 8'}])
        ]),
         
    )
    
])


layout.update(dict(updatemenus=updatemenus))

fig = go.Figure(data=data1, layout=layout)

              
py.iplot(fig, filename = "Percentage of Completed offers by Age")

In [ ]:
fig.update_layout(
    updatemenus=[
        go.layout.Updatemenu(
            buttons=list([
                dict(
                    args=["colorscale", "Viridis"],
                    label="Viridis",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Cividis"],
                    label="Cividis",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Blues"],
                    label="Blues",
                    method="restyle"
                ),
                dict(
                    args=["colorscale", "Greens"],
                    label="Greens",
                    method="restyle"
                ),
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
        go.layout.Updatemenu(
            buttons=list([
                dict(
                    args=["reversescale", False],
                    label="False",
                    method="restyle"
                ),
                dict(
                    args=["reversescale", True],
                    label="True",
                    method="restyle"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.37,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
        go.layout.Updatemenu(
            buttons=list([
                dict(
                    args=[{"contours.showlines": False, "type": "contour"}],
                    label="Hide lines",
                    method="restyle"
                ),
                dict(
                    args=[{"contours.showlines": True, "type": "contour"}],
                    label="Show lines",
                    method="restyle"
                ),
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.58,
            xanchor="left",
            y=button_layer_1_height,
            yanchor="top"
        ),
    ]
)

The plot illustrates the Percentages of Completed offers by each age group that can be filtered by offer. For example, at the age group 35 years old, offer 6 has the highest completion rate 82.61%. While the age group 25 years old prefer offer 5 the most with a completion rate of 77.55%.

### Bonus

In [191]:
# plotting a Sunburst Charts shows the numbers of customers
# with respect to all transactions where the the customers completed an offer 

trace = go.Sunburst(
    labels=["Transactions",
            "BOGO", "Discount",
            "Offer 1", "Offer 2", "Offer 3", "Offer 4", "Offer 5", 
            "Offer 6", "Offer 7", "Offer 8",
            "Female", "Male", "Other", 
            "Female", "Male", "Other",
            "Female", "Male", "Other", 
            "Female", "Male", "Other", 
            "Female", "Male", "Other", 
            "Female", "Male", "Other", 
            "Female", "Male", "Other", 
            "Female", "Male", "Other"],
    parents=["", 
             "Transactions", "Transactions", 
             "BOGO", "BOGO", "BOGO", "Discount", "Discount", 
             "Discount", "Discount", "BOGO",
             "Offer 1", "Offer 1", "Offer 1",
             "Offer 2", "Offer 2", "Offer 2", 
             "Offer 3", "Offer 3", "Offer 3",
             "Offer 4", "Offer 4", "Offer 4",
             "Offer 5", "Offer 5", "Offer 5",
             "Offer 6", "Offer 6", "Offer 6",
             "Offer 7", "Offer 7", "Offer 7",
             "Offer 8", "Offer 8", "Offer 8"],
    values=[26226, 
            9937, 12089,
            2573, 2773, 2374, 2499, 3729, 
            2592, 3269, 1917,
            1197, 1324, 52,
            1281, 1441, 51,
            1134, 917, 323,
            1247, 1252, 0,
            1626, 2052, 51,
            1188, 1354, 50,
            1371, 1755, 141,
            872, 1045, 0],
    branchvalues="total",
    outsidetextfont = {"size": 15, "color": "#377eb8"},
    marker = {"line": {"width": 2}})

layout = go.Layout(
    title = 'test',
    margin = go.layout.Margin(t=0, l=0, r=0, b=0))

py.iplot(go.Figure([trace], layout), filename='basic_sunburst_chart_total_branchvalues')